<a href="https://colab.research.google.com/github/mugdha711/MLAP-Class/blob/master/3B)_COLLABORATIVE_RECOMMENDATION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MOVIE DATASET

###uPLOAD DATA

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
rating_df = pd.read_csv( r"/content/ratings.csv" )
rating_df.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [2]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [4]:
#timestamp not required so drop
rating_df.drop( 'timestamp', axis = 1, inplace = True )

In [5]:
  len( rating_df.userId.unique() ) #no. of unique users

610

In [6]:
len( rating_df.movieId.unique() ) #no. of unoque movies

9724

##**User Based Similarity Approach**

###Creating Matrix

In [7]:
#create a pivot table or matrix and represent users as rows and movies as columns.

user_movies_df = rating_df.pivot( index='userId',columns='movieId',values = "rating" ).reset_index(drop=True)
user_movies_df.index = rating_df.userId.unique()
user_movies_df.iloc[0:5, 0:15]

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#replace NAN with 0

user_movies_df.fillna( 0, inplace = True )
user_movies_df.iloc[0:5, 0:10]

movieId,1,2,3,4,5,6,7,8,9,10
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


###Calculating Cosine Similarity between Users

In [9]:
#compute the similarity between rows, it will represent the similarity between those users. sklearn.metrics.pairwise_distances can be used to compute distance between all pairs of users
#Cosine similarity closer to 1 means users are very similar and closer to 0 means users are very dissimilar.

In [10]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
user_sim = 1 - pairwise_distances( user_movies_df.values, metric="cosine" )
#Store the results in a dataframe
user_sim_df = pd.DataFrame( user_sim )
# set the index and column names to user ids (0 to 610)
user_sim_df.index = rating_df.userId.unique()
user_sim_df.columns = rating_df.userId.unique()

In [11]:
user_sim_df.iloc[0:5, 0:5]

,1,2,3,4,5
1,1.000000,0.027283,0.059720,0.194395,0.129080
2,0.027283,1.000000,0.000000,0.003726,0.016614
3,0.059720,0.000000,1.000000,0.002251,0.005020
4,0.194395,0.003726,0.002251,1.000000,0.128659
5,0.129080,0.016614,0.005020,0.128659,1.000000


In [12]:
user_sim_df.shape

(610, 610)

In [17]:
user_sim_df.iloc[:5,:5]

,1,2,3,4,5
1,1.000000,0.027283,0.059720,0.194395,0.129080
2,0.027283,1.000000,0.000000,0.003726,0.016614
3,0.059720,0.000000,1.000000,0.002251,0.005020
4,0.194395,0.003726,0.002251,1.000000,0.128659
5,0.129080,0.016614,0.005020,0.128659,1.000000


In [18]:
np.fill_diagonal( user_sim, 0 ) #becasue these are same items, we want to look at different items thus set them as 0
user_sim_df.iloc[0:5, 0:5]      #helps to avoid selecting self as the most similar user.

,1,2,3,4,5
1,0.000000,0.027283,0.059720,0.194395,0.129080
2,0.027283,0.000000,0.000000,0.003726,0.016614
3,0.059720,0.000000,0.000000,0.002251,0.005020
4,0.194395,0.003726,0.002251,0.000000,0.128659
5,0.129080,0.016614,0.005020,0.128659,0.000000


###Filtering Similar Users

In [19]:
#to get the most similar user to the userids. here display just for irst 5 user ids
user_sim_df.idxmax(axis=1)[0:5] 

1    266
2    366
3    313
4    391
5    470
dtype: int64

In [20]:
#to check and understand whatis it checking - do for userid 2

user_sim_df.iloc[1:2, 330:340]

,331,332,333,334,335,336,337,338,339,340
2,0.199366,0.073652,0.050674,0.053668,0.073991,0.046544,0.018408,0.074145,0.111447,0.03063


###Loading the movies dataset

In [21]:
movies_df = pd.read_csv( r"/content/movies.csv" )
movies_df[0:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
movies_df.drop( 'genres', axis = 1, inplace = True ) #genre not required as of now    

###Find common movies of similar users


In [23]:
def get_user_similar_movies( user1, user2 ):
# Inner join between movies watched between two users will give the common movies watched.
    common_movies = rating_df[rating_df.userId == user1].merge(rating_df[rating_df.userId == user2],on = "movieId",how = "inner" )
# join the above result set with movies details
    return common_movies.merge( movies_df, on = 'movieId' )

In [24]:
common_movies = get_user_similar_movies( 2, 330 )
common_movies[(common_movies.rating_x >= 4.0) & ((common_movies.rating_y >= 4.0))]

,userId_x,movieId,rating_x,userId_y,rating_y,title
1,2,3578,4.0,330,4.0,Gladiator (2000)
2,2,6874,4.0,330,5.0,Kill Bill: Vol. 1 (2003)
3,2,48516,4.0,330,4.5,"Departed, The (2006)"
4,2,58559,4.5,330,5.0,"Dark Knight, The (2008)"
5,2,68157,4.5,330,5.0,Inglourious Basterds (2009)
7,2,79132,4.0,330,5.0,Inception (2010)


In [25]:
common_movies = get_user_similar_movies( 2, 338 )
common_movies

,userId_x,movieId,rating_x,userId_y,rating_y,title
0,2,318,3.0,338,5.0,"Shawshank Redemption, The (1994)"
1,2,6874,4.0,338,4.5,Kill Bill: Vol. 1 (2003)


**Challenges in User similarity approach-**
We need to wait until the new user buys a few items and rates them. Only then users with similar preferences can be found and recommendations can be made based on that. This is called **cold start** problem in recommender systems

##**Item BAsed Similarity Approach**

###Calculating Cosine Similarity between movies

In [26]:
rating_mat = rating_df.pivot( index='movieId',columns='userId',values = "rating" ).reset_index(drop=True)
# fill all NaNs with 0
rating_mat.fillna( 0, inplace = True )
# Find the correlation between movies
movie_sim = 1 - pairwise_distances( rating_mat.values,metric="correlation" )
# Fill the diagonal with 0, as it repreresent the auto-correlation of movies
movie_sim_df = pd.DataFrame( movie_sim )

In [27]:
movie_sim_df.iloc[0:5, 0:5]

,0,1,2,3,4
0,1.000000,0.231327,0.173213,-0.028917,0.192474
1,0.231327,1.000000,0.191945,0.071269,0.200526
2,0.173213,0.191945,1.000000,0.067143,0.370171
3,-0.028917,0.071269,0.067143,1.000000,0.167910
4,0.192474,0.200526,0.370171,0.167910,1.000000


In [28]:
movie_sim_df.shape

(9724, 9724)

###Finding most similar movies

In [29]:
def get_similar_movies( movieid, topN = 5 ):
    movieidx = movies_df[movies_df.movieId == movieid].index[0]
    movies_df['similarity'] = movie_sim_df.iloc[movieidx]
    top_n = movies_df.sort_values( ["similarity"], ascending = False )[0:topN]
    return top_n

###Finding similar movies to Godfather

In [31]:
movies_df[movies_df.movieId == 858] #gives id of godfather

,movieId,title
659,858,"Godfather, The (1972)"


In [32]:
get_similar_movies(858)

,movieId,title,similarity
659,858,"Godfather, The (1972)",1.000000
921,1220,"Blues Brothers, The (1980)",0.769390
913,1212,"Third Man, The (1949)",0.560246
895,1192,Paris Is Burning (1990),0.496048
827,1088,Dirty Dancing (1987),0.442128


##Using Surprise Library - pip install scikit-surprise

In [ ]:
#!pip install -c conda-forge mlxtend
#update scikit-surprise

#conda install --channel conda-forge scikit-surprise


In [58]:
from surprise import Dataset, Reader, KNNBasic,accuracy
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

DistributionNotFound: ignored

#Installing Conda

In [43]:
!which python # should return /usr/local/bin/python

/usr/local/bin/python


In [44]:
!python --version

Python 3.6.9


In [45]:
!echo $PYTHONPATH

/env/python


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [47]:
!which conda # should return /usr/local/bin/conda

/usr/local/bin/conda


In [48]:
!conda --version # should return 4.5.4

conda 4.5.4


In [49]:
!which python # still returns /usr/local/bin/python

/usr/local/bin/python


In [50]:
!python --version # now returns Python 3.6.5 :: Anaconda, Inc.

Python 3.6.5 :: Anaconda, Inc.


In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

In [52]:
!conda --version # now returns 4.8.3

conda 4.8.4


In [53]:
  import sys
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
!ls /usr/local/lib/python3.6/dist-packages

In [55]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

In [ ]:
!conda install -c conda-forge mlxtend --yes

!conda install --channel conda-forge scikit-surprise --yes


# Surprise

In [71]:
#!pip install scikit-surprise
#import sys
#_ = (sys.path
#        .append("/usr/local/lib/python3.6/site-packages"))
#!conda install --channel conda-forge scikit-surprise --yes
#!conda install -c conda-forge mlxtend --yes
#from /usr/local/lib/python3.6/site-packages/surprise import Dataset  
#%%bash
#scikit-surprise


bash: line 1: locate: command not found


In [72]:
!ls /usr/local/lib/python3.6/dist-packages | scikit-surprise

/bin/bash: scikit-surprise: command not found
